<h3>-----------------------------Stock Anaylsis Parsing Code-----------------------------<h>
<h6>url = 'https://stockanalysis.com/actions/splits/'<h>

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

# URL of the webpage that contains stock split data
url = 'https://stockanalysis.com/actions/splits/'

# Custom headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Making the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the table or div that contains the stock split data
    table = soup.find('table')  # Assuming the data is in a table
    if table:
        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:  # Skipping the header row
            cells = row.find_all('td')
            if len(cells) > 1:
                date_str = cells[0].text.strip()
                date = datetime.strptime(date_str, "%b %d, %Y").strftime("%-m/%-d/%Y")
                symbol = cells[1].text.strip()
                company_name = cells[2].text.strip()
                split_type = cells[3].text.strip()
                split_ratio = cells[4].text.strip().replace(" for ", " : ")
                data.append([date, symbol, company_name, split_type, split_ratio])
        
        # Creating DataFrame
        df = pd.DataFrame(data, columns=['Date', 'Symbol', 'Company Name', 'Type', 'Split Ratio'])
        
        # Removing rows that contain "Forward" in the 'Type' column
        df = df[~df['Type'].str.contains("Forward", case=False, na=False)]
        
        # Saving DataFrame to CSV
        df.to_csv('/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/StockAnalysis_Split_Data.csv', index=False)
        print("Data has been saved to StockAnalysis_Split_Data.csv")
    else:
        print("Failed to find the table containing stock split data.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Data has been saved to StockAnalysis_Split_Data.csv
